# Research around Voice Authentication

In [1]:
import os
import numpy as np
import librosa
import pyaudio
import wave
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

In [2]:
sample_rate = 22050
duration = 5 
num_mfcc = 13
num_recordings = 3
model_path = "voice_auth_model.pkl"

In [3]:
def record_audio(filename, duration=duration, sample_rate=sample_rate):
    """Record audio from the microphone and save to a file."""
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1

    p = pyaudio.PyAudio()
    stream = p.open(format=format, channels=channels, rate=sample_rate, input=True, frames_per_buffer=chunk)

    print(f"Recording: {filename}")
    frames = []

    for _ in range(0, int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(format))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

In [4]:
def extract_features(file_path):
    """Extract MFCC features from an audio file."""
    audio, sr = librosa.load(file_path, sr=sample_rate)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc)
    return np.mean(mfccs.T, axis=0)

In [5]:
def train_model(features):
    """Train a GMM model on the extracted features."""
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    model = GaussianMixture(n_components=1, covariance_type='diag', max_iter=200, random_state=42)
    model.fit(features_scaled)
    return model, scaler

In [6]:
def save_model(model, scaler, path=model_path):
    """Save the trained model and scaler to a file."""
    import pickle
    with open(path, 'wb') as f:
        pickle.dump((model, scaler), f)

In [7]:
def load_model(path=model_path):
    """Load the trained model and scaler from a file."""
    import pickle
    with open(path, 'rb') as f:
        return pickle.load(f)

In [8]:
def authenticate_voice(file_path, model, scaler):
    """Authenticate a voice sample against the trained model."""
    features = extract_features(file_path)
    features_scaled = scaler.transform([features])
    score = model.score(features_scaled)
    return score

In [ ]:
def main():
    # Record initial samples for training
    training_files = [f"voice_sample_{i}.wav" for i in range(num_recordings)]
    for file in training_files:
        record_audio(file)

    # Extract features and train model
    features = np.array([extract_features(file) for file in training_files])
    model, scaler = train_model(features)
    save_model(model, scaler)

    # Authenticate a new voice sample
    test_file = "test_voice_sample.wav"
    record_audio(test_file)
    model, scaler = load_model()
    score = authenticate_voice(test_file, model, scaler)

    # Threshold for authentication (this is a simple example, adjust as needed)
    threshold = -50
    if score > threshold:
        print("Authentication successful: Same person")
    else:
        print("Authentication failed: Different person")

if __name__ == "__main__":
    main()

Recording: voice_sample_0.wav
Recording: voice_sample_1.wav
Recording: voice_sample_2.wav
